In [1]:
import numpy as np

In [22]:
B = 0.7
M = np.array([[0.0, 0.5, 1.0, 0.2, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.2, 0.0, 0.0],
              [0.0, 0.5, 0.0, 0.2, 1/3, 0.0],
              [0.0, 0.0, 0.0, 0.0, 1/3, 0.5],
              [0.0, 0.0, 0.0, 0.2, 0.0, 0.5],
              [0.0, 0.0, 0.0, 0.2, 1/3, 0.0]])

In [25]:
def page_rank(M, tax, n_iter=100, E=None):
    """
    Parameters
    ---------
        M : array of shape=(n, n), connectivity matrix.
        tax : amound of tax, for example 0.8
        E : epsilon, min difference between previous and current rank vector to proceed the next iteration
    """
    r = np.ones(shape=M.shape[0])# / M.shape[0] # uncoment if initialization with 1/n
    r_new = np.zeros_like(r)
    e = np.ones_like(r) / M.shape[0]
    
    for i in range(n_iter):
        r_new = tax * M@r + (1-tax) * e
        print(f'After iter {i+1}: {r}')
        
        # check convergance
        if E is not None:
            if len(np.nonzero(r - r_new < E)[0]) == len(r):
                print(f'Converged after {i+1} iterations')
                break
        
        r = r_new
        
    return r

In [26]:
r_res = page_rank(M, tax=1.0, n_iter=5)

After iter 1: [ 1.  1.  1.  1.  1.  1.]
After iter 2: [ 1.7         0.2         1.03333333  0.83333333  0.7         0.53333333]
After iter 3: [ 1.3         0.16666667  0.5         0.5         0.43333333  0.4       ]
After iter 4: [ 0.68333333  0.1         0.32777778  0.34444444  0.3         0.24444444]
After iter 5: [ 0.44666667  0.06888889  0.21888889  0.22222222  0.19111111  0.16888889]


In [27]:
a, b, c = r_res[0], r_res[1], r_res[2]

In [28]:
print(b, 0.575*a + 0.15*c)
print(0.95*a, 0.9*c + 0.05*b)
print(0.85*c, b + 0.575*a)
print(a, c + 0.15*b)

0.0444444444444 0.192611111111
0.282888888889 0.130555555556
0.121203703704 0.215666666667
0.297777777778 0.149259259259


In [29]:
def ts_page_rank(M, tax, S=None, n_iter=1, E=None):
    """
    Parameters
    ---------
        M : array of shape=(n, n), connectivity matrix.
        tax : amound of tax, 
            that is the probability of random walking. 1-tax is the probability that it jumps to some node in S.
        S : array of shape=(n,), with ones indicating nodes in topic set.
        n_iter : int, number of iterations
        E : epsilon, min difference between previous and current rank vector to proceed the next iteration
    """
    r = np.ones(shape=M.shape[0]) / M.shape[0]
    r_new = np.zeros_like(r)
    
    if S is None:
        S = np.ones(shape=M.shape[0])
    S = S / np.sum(S)
    
    for i in range(n_iter):
        r_new = tax * M@r + (1-tax) * S
        print(f'After iter {i+1}: {r_new}')
        
        # check convergance
        if E is not None:
            if len(np.nonzero(r - r_new < E)[0]) == len(r):
                print(f'Converged after {i+1} iterations')
                break
        
        r = r_new
        
    return r

In [30]:
r_res = ts_page_rank(M, tax=0.85, n_iter=10, E=0.001)

After iter 1: [ 0.26583333  0.05333333  0.17138889  0.14305556  0.12416667  0.10055556]
After iter 2: [ 0.21766667  0.04931944  0.10716667  0.10291667  0.09205556  0.0845    ]
After iter 3: [ 0.15454826  0.04249583  0.089539    0.08699491  0.07840833  0.06857824]
After iter 4: [ 0.13395802  0.03978913  0.08006556  0.07636145  0.06893489  0.06200483]
After iter 5: [ 0.12294755  0.03798145  0.07442338  0.0708836   0.0643335   0.057513  ]
After iter 6: [ 0.1164522   0.03705021  0.07142015  0.06767085  0.06149324  0.05527804]
After iter 7: [ 0.11295751  0.03650404  0.06967347  0.06591625  0.05999721  0.05392713]
After iter 8: [ 0.11094243  0.03620576  0.06871919  0.06491824  0.05912479  0.05320497]
After iter 9: [ 0.10983486  0.0360361   0.06817557  0.06436414  0.05864821  0.05278812]
After iter 10: [ 0.10920648  0.0359419   0.06787424  0.06405195  0.05837686  0.0525589 ]
Converged after 10 iterations


In [10]:
def hubs_authorities(M, tax, n_iter=1, E=None):
    n = M.shape[0]
    a = np.ones(shape=n)
    h = np.ones(shape=n)
    e = np.ones(shape=n) / n
    # initialize
    a = a / np.sqrt(n)
    h = h / np.sqrt(n)
    A = tax * M + (1-tax) * e
    
    for i in range(n_iter):
        # calc new values
        h_new = A@a
        a_new = A.T@h
        
        # normalize
        h_new = h_new / np.sum(h_new)
        a_new = a_new / np.sum(a_new)
         
        print(f'After iter {i+1}: hub scores: {h_new}, auth scores: {a_new}')
        
        # check convergance
        if E is not None:
            if len(np.nonzero((h - h_new)**2 < E)[0]) == len(h) and len(np.nonzero((a - a_new)**2 < E)[0]) == len(a):
                print(f'Converged after {i+1} iterations')
                break
        
        h = h_new
        a = a_new
        
    return h, a
    

In [11]:
tax = 1.0
M = np.array([[1.0, 1.0, 1.0],
              [1.0, 0.0, 1.0],
              [0.0, 1.0, 0.0]])

hubs_authorities(M, tax=tax, n_iter=3, E=0.01)

After iter 1: hub scores: [ 0.5         0.33333333  0.16666667], auth scores: [ 0.33333333  0.33333333  0.33333333]
After iter 2: hub scores: [ 0.5         0.33333333  0.16666667], auth scores: [ 0.35714286  0.28571429  0.35714286]
Converged after 2 iterations


(array([ 0.5       ,  0.33333333,  0.16666667]),
 array([ 0.33333333,  0.33333333,  0.33333333]))

# Pearson

In [12]:
import numpy as np
from scipy.spatial.distance import cosine

def pearson(u, v):
    non_missing_u = np.nonzero(u > 0)[0]
    non_missing_v = np.nonzero(v > 0)[0]
    
    avg_u = np.sum(u) / len(non_missing_u)
    avg_v = np.sum(v) / len(non_missing_v)
    print(f'avg_u: {avg_u}')
    print(f'avg_v: {avg_v}')
    
    u[non_missing_u] = u[non_missing_u] - avg_u
    v[non_missing_v] = v[non_missing_v] - avg_v
    print(f'u: {u}')
    print(f'v: {v}')
    return 1 - cosine(u, v)

In [13]:
a = np.array([4.0, 3.0, 4.0, 0.0, 0.0, 1.0])
b = np.array([4.0, 1.0, 2.0, 2.0, 0.0, 1.0])
pearson(a, b)

avg_u: 3.0
avg_v: 2.0
u: [ 1.  0.  1.  0.  0. -2.]
v: [ 2. -1.  0.  0.  0. -1.]


0.66666666666666663

In [14]:
a = np.array([4.0, 3.0, 4.0, 0.0, 0.0, 1.0])
c = np.array([4.0, 0.0, 0.0, 3.0, 1.0, 0.0])
pearson(a, c)

avg_u: 3.0
avg_v: 2.6666666666666665
u: [ 1.  0.  1.  0.  0. -2.]
v: [ 1.33333333  0.          0.          0.33333333 -1.66666667  0.        ]


0.25197631533948495

# Text mining

TF

In [15]:
TF = np.array([[1, 0, 1, 0, 1, 0, 1, 0, 0],
               [1, 0, 0, 0, 1, 1, 1, 1, 0],
               [0, 0, 0, 0, 1, 1, 1, 0, 1],
               [1, 1, 0, 1, 1, 0, 0, 0, 1]])

q = np.array([0, 0, 0, 0, 0, 1, 1, 0, 0])

In [16]:
def tf_cos(TF, q):
    for i in range(len(TF)):
        print(1 - cosine(TF[i], q))

In [17]:
tf_cos(TF, q)

0.353553390593
0.632455532034
0.707106781187
0.0


TF-IDF

In [18]:
import numpy as np

N = TF.shape[0]
n = np.array([np.sum(TF[:, i]) for i in range(TF.shape[1])])

TF_IDF = np.ones_like(TF, dtype=np.float32)

for i in range(TF.shape[0]):
    for j in range(TF.shape[1]):
        
        tf = TF[i, j]
        idf = np.log(N / n[j])
        
        TF_IDF[i, j]= round(tf * idf, 3)

        
TF_IDF

array([[ 0.28799999,  0.        ,  1.38600004,  0.        ,  0.        ,
         0.        ,  0.28799999,  0.        ,  0.        ],
       [ 0.28799999,  0.        ,  0.        ,  0.        ,  0.        ,
         0.69300002,  0.28799999,  1.38600004,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.69300002,  0.28799999,  0.        ,  0.69300002],
       [ 0.28799999,  1.38600004,  0.        ,  1.38600004,  0.        ,
         0.        ,  0.        ,  0.        ,  0.69300002]], dtype=float32)

In [19]:
q_tfidf = np.array([q[i] * np.log(N / n[i]) for i in range(len(q))], dtype=np.float32)
q_tfidf

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.69314718,  0.28768209,  0.        ,  0.        ], dtype=float32)

In [20]:
def tfidf_cos(TF_IDF, q_tfidf):

    for i in range(len(TF_IDF)):
        
        print(1 - cosine(TF_IDF[i], q_tfidf))

In [21]:
tfidf_cos(TF_IDF, q_tfidf)

0.0764221847057
0.468386530876
0.73467361927
0.0
